### Import des bibliothéques
---

In [22]:
import pandas as pd
import requests as rq
import json as js

### Chargement des données
---

In [23]:
df_ville = pd.read_csv('df_ville.csv')
df_ville.head()

Unnamed: 0              Ville   Latitude  Longitude
0           0  Mont Saint Michel  48.690680   5.882649
1           1            St Malo  48.649518  -2.026041
2           2             Bayeux  49.276462  -0.702474
3           3           Le Havre  49.493898   0.107973
4           4              Rouen  49.440459   1.093966

### Scraping méteo Site openweathermap
---
#### prévisions météorologiques à 5 jours - données météorologiques par pas de 3 heures

In [24]:
# Site Openweathermap : https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&cnt=40&appid={API_KEY}&units=metric&lang=fr"
   
API_KEY = 'b9b15b3ef33eb08f185286b5682ad89d'
URL = 'https://api.openweathermap.org/data/2.5/forecast'


### Uniformiser les noms de colonnes 

In [25]:
# fonction nom de ville du df_ville identique au nom de ville API méteo Openweather ?

def check_city(city_name):
    """Retourne True si la ville est reconnue par l'API, False sinon"""
    params = {'q': f'{city_name},FR', 'appid': API_KEY}
    response = rq.get(
        URL, 
        params=params)
    
    data = response.json()
    if data.get('cod') == '200':
        return True
    else:
        return False

df_ville['ville_valide'] = df_ville['Ville'].apply(check_city)

print("\n--- Nombre de ville True or False ---\n")
print(df_ville['ville_valide'].value_counts())
print(f"\n{(~df_ville['ville_valide']).sum()} noms de ville ne sont pas corrects") #~ inverse le booléan : false

# création df des villes non valides
df_ville_false = df_ville[df_ville['ville_valide']==False]
df_ville_false.head(6)



--- Nombre de ville True or False ---

ville_valide
True     28
False     6
Name: count, dtype: int64

6 noms de ville ne sont pas corrects


Unnamed: 0                         Ville   Latitude  Longitude  \
0            0             Mont Saint Michel  48.690680   5.882649   
1            1                       St Malo  48.649518  -2.026041   
9            9  Chateau du Haut Koenigsbourg  48.249523   7.345492   
17          17            Bormes les Mimosas  43.150697   6.341928   
20          20               Aix en Provence  43.529842   5.447474   
25          25      Saintes Maries de la Mer  43.451592   4.427720   

    ville_valide  
0          False  
1          False  
9          False  
17         False  
20         False  
25         False

In [26]:
# Récupérer avec la lat et lon, les noms de ville valide de l'API méteo Openweather
villes_correctes = []

for i, row in df_ville.iterrows() :
    lat = row['Latitude']
    lon = row['Longitude']

    try:
        params = {'lat': lat, 'lon': lon, 'appid': API_KEY}

        response = rq.get(
            URL,
            params=params, 
            timeout=10 # si le site ne répond pas s'arrête ds les 10s
        )
        response.raise_for_status()
        data = response.json()

        Villes = data['city']['name']
        villes_correctes.append(Villes)

    except rq.RequestException as e:
        print(f"\tErreur pour {row['Ville']} : {e}")  
        villes_correctes.append(None)  # si erreur, on met None

# Ajouter la colonne d’un coup
df_ville['Villes'] = villes_correctes
df_ville=df_ville.drop(columns=['Ville', 'ville_valide'])
df_ville.head()

Unnamed: 0   Latitude  Longitude    Villes
0           0  48.690680   5.882649      Toul
1           1  48.649518  -2.026041   St-Malo
2           2  49.276462  -0.702474    Bayeux
3           3  49.493898   0.107973  Le Havre
4           4  49.440459   1.093966     Rouen

### Scraper les infos méteo : température, 4 journées entiéres, tendance de la matinée et aprés-midi

In [32]:
# essai date
import pandas as pd
import requests as rq
from datetime import datetime

API_KEY = 'b9b15b3ef33eb08f185286b5682ad89d'
URL = 'https://api.openweathermap.org/data/2.5/forecast'

# Initialiser le DataFrame vide (colonnes dynamiques ajoutées plus tard)
df_ville_meteo = pd.DataFrame(columns=['Ville', 'Latitude', 'Longitude'])

# Boucle sur chaque ville
for i, row in df_ville.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    ville = row['Villes']
    
    new_row = {'Ville': ville, 'Latitude': lat, 'Longitude': lon}
    
    try:
        response = rq.get(
            URL,
            params={'lat': lat, 'lon': lon, 'appid': API_KEY, 'units': 'metric', 'lang': 'fr'},
            timeout=10
        )
        response.raise_for_status()
        data = response.json()
        
        if 'list' in data:
            # Dictionnaire pour stocker la météo par date
            daily_forecast = {}
            
            for forecast in data['list']:
                dt_txt = forecast['dt_txt']  # date et heure ss forme de text ex : "2025-09-30 09:00:00"
                date_str, hour = dt_txt.split(" ") # on sépare la chaine en 2 morecaux date_str : "2025-09-30" et hour:"09:00:00"
                
                # Transformer la date en format jj/mm/aaaa
                date_format = datetime.strptime(date_str, "%Y-%m-%d").strftime("%d/%m/%Y")
                
                # Se concentrer sur 09:00 et 15:00 données ttes les 3h
                if hour in ["09:00:00", "15:00:00"]:
                    temp = round(forecast['main']['temp']) # arroundi la températire 
                    desc = forecast['weather'][0]['description']
                    
                    # Gestion pluie/neige
                    pluie = 0
                    if 'rain' in forecast and '3h' in forecast['rain']:
                        pluie = forecast['rain']['3h']
                    elif 'snow' in forecast and '3h' in forecast['snow']:
                        pluie = forecast['snow']['3h']
                    
                    cell_value = f"{desc} ({temp}°C)"
                    
                    # Remplir le dictionnaire selon heure
                    if date_format not in daily_forecast:
                        daily_forecast[date_format] = {} # verification si la date du jour existe déjà dnas le daily_forecast
                    if hour == "09:00:00":
                        daily_forecast[date_format]['matin'] = cell_value
                    elif hour == "15:00:00":
                        daily_forecast[date_format]['aprem'] = cell_value
            
            # Ajouter les colonnes dynamiques avec les vraies dates
            sorted_dates = sorted(daily_forecast.keys())[:4]  # max 4 jours, trié par ordre croissance avec sorted
            for date in sorted_dates:
                new_row[f'Matin {date}'] = daily_forecast[date].get('matin', '')# création colonne avec date + matin
                new_row[f'Aprés-midi {date}'] = daily_forecast[date].get('aprem', '')# idem + aprem
            
            print(f"Météo récupérée pour {ville}")
        else:
            print(f"Aucune prévision disponible pour {ville}")
        
    except rq.RequestException as e:
        print(f"Erreur pour {ville}: {e}")
    
    # Ajouter la ligne au DataFrame
    df_ville_meteo = pd.concat([df_ville_meteo, pd.DataFrame([new_row])], ignore_index=True)


Météo récupérée pour Toul


C:\Users\natha\AppData\Local\Temp\ipykernel_24728\2445673376.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_ville_meteo = pd.concat([df_ville_meteo, pd.DataFrame([new_row])], ignore_index=True)


Météo récupérée pour St-Malo
Météo récupérée pour Bayeux
Météo récupérée pour Le Havre
Météo récupérée pour Rouen
Météo récupérée pour Palais-Royal
Météo récupérée pour Amiens
Météo récupérée pour Lille
Météo récupérée pour Arrondissement de Strasbourg
Météo récupérée pour Saint-Hippolyte
Météo récupérée pour Colmar
Météo récupérée pour Eguisheim
Météo récupérée pour Besançon
Météo récupérée pour Larrey
Météo récupérée pour Annecy
Météo récupérée pour Grenoble
Météo récupérée pour Vieux Lyon
Météo récupérée pour Bormes-les-Mimosas
Météo récupérée pour Cassis
Météo récupérée pour Marseille
Météo récupérée pour Aix-en-Provence
Météo récupérée pour Avignon
Météo récupérée pour Uzès
Météo récupérée pour Nîmes
Météo récupérée pour Aigues-Mortes
Météo récupérée pour Saintes-Maries-de-la-Mer
Météo récupérée pour Collioure
Météo récupérée pour Carcassonne
Météo récupérée pour Tarascon-sur-Ariège
Météo récupérée pour Toulouse
Météo récupérée pour Montauban
Météo récupérée pour Biarritz
Météo ré

In [33]:
df_ville_meteo.head()

Ville   Latitude  Longitude    Matin 01/10/2025  \
0      Toul  48.690680   5.882649  ciel dégagé (12°C)   
1   St-Malo  48.649518  -2.026041      couvert (15°C)   
2    Bayeux  49.276462  -0.702474      couvert (15°C)   
3  Le Havre  49.493898   0.107973      nuageux (14°C)   
4     Rouen  49.440459   1.093966  peu nuageux (16°C)   

          Aprés-midi 01/10/2025    Matin 02/10/2025  \
0  partiellement nuageux (15°C)  peu nuageux (11°C)   
1                nuageux (19°C)      couvert (16°C)   
2                couvert (19°C)      couvert (16°C)   
3                couvert (16°C)      couvert (15°C)   
4                couvert (20°C)      couvert (15°C)   

          Aprés-midi 02/10/2025 Matin 03/10/2025 Aprés-midi 03/10/2025  \
0  partiellement nuageux (14°C)   couvert (11°C)        couvert (14°C)   
1                couvert (21°C)   couvert (17°C)   légère pluie (19°C)   
2                couvert (22°C)   couvert (17°C)        couvert (18°C)   
3                couvert (20°C)   couvert (17°C)   légère pluie (17°C)   
4                couvert (20°C)   couvert (15°C)   légère pluie (16°C)   

       Matin 04/10/2025 Aprés-midi 04/10/2025  
0  pluie modérée (12°C)   légère pluie (13°C)  
1   légère pluie (14°C)   légère pluie (14°C)  
2    ciel dégagé (14°C)   légère pluie (13°C)  
3   légère pluie (14°C)   légère pluie (14°C)  
4   légère pluie (14°C)   légère pluie (14°C)

### Sauvegarde df_meteo
---

In [34]:
df_ville_meteo.to_csv('df_ville_meteo.csv', index=False, encoding='utf-8-sig')
